In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from sklearn.model_selection import cross_validate, StratifiedKFold
from tqdm import tqdm
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import warnings
import os
import sys
import copy

In [2]:
def evaluator(y_pred, y_test, verbose=False):
    """Returns evaluation metric scores"""
    accuracy = accuracy_score(y_pred=y_pred, y_true=y_test)
    balanced_accuracy = balanced_accuracy_score(y_pred=y_pred, y_true=y_test)
    f1 = f1_score(y_pred=y_pred, y_true=y_test, average='weighted')
    recall = recall_score(y_pred=y_pred, y_true=y_test, average='weighted')
    precision = precision_score(y_pred=y_pred, y_true=y_test, average='weighted')
    confusion = confusion_matrix(y_pred=y_pred, y_true=y_test)

    # display scores
    if verbose:
        ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=[False, True]).plot(cmap=plt.cm.Blues)
        plt.title('Physical fatigue')

        print(f'accuracy: {accuracy}\n'
              f'balanced accuracy: {balanced_accuracy}\n'
              f'f1 (weighted): {f1}\n'
              f'recall (weighted): {recall}\n'
              f'precision (weighted): {precision}')

    return {'accuracy': accuracy,
            'balanced_accuracy': balanced_accuracy,
            'f1': f1,
            'recall': recall,
            'precision': precision,
            'confusion': confusion}

In [3]:
VARIABLES = ['ActivityCounts', 'Barometer', 'BloodPerfusion',
             'BloodPulseWave', 'EnergyExpenditure', 'GalvanicSkinResponse', 'HR',
             'HRV', 'RESP', 'Steps', 'SkinTemperature', 'ActivityClass']

In [4]:
NORMALIZE_TRAIN = True # whether to normalize data acc. to training data
SHUFFLE = True # whether to shuffle data before applying CV

In [5]:
# for reproducability
SEED = 42
SEED2 = 0

# Import data

In [6]:
# file path to data folder
path = './Output'

In [7]:
N, FEATURES = sum([1 for p in os.listdir(path) if p[:19] == 'feature_vector_stat']), \
              *np.load(path + '/feature_vector_stat0.npy').shape
print(f'datapoints: {N}, features: {FEATURES}')

datapoints: 317, features: 284


Feature vector, labels

In [8]:
# init
X = np.empty((N, FEATURES))
y = np.empty((N, 2))

# load individual datapoints
for i in range(N):
    X[i, ] = np.load(path + f'/feature_vector_stat{i}.npy', allow_pickle=True)
    y[i, ] = np.load(path + f'/labels_stat{i}.npy', allow_pickle=True)

Metadata (subjectID etc.)

In [9]:
with open(path + '/metadata_stat.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

# Random Forest

### 5-fold stratified CV

In [10]:
# separate label prediction
y_phf, y_mf = y[:, 0], y[:, 1]

In [18]:
# CV ranges
n_trees = [3, 10, 50, 100, 300, 1000]
max_features = ['auto', 'sqrt', 'log2']
max_depths = [10, 30, 50, 100]
criterions = ['gini', 'entropy']
min_samples_splits = [2, 5, 10]

In [12]:
%%time
# nested CV
folds = 5

with warnings.catch_warnings():
    # ignore sklearn warning
    warnings.filterwarnings('ignore')

    for fatigue in ('Physical fatigue', 'Mental fatigue'):
        # load labels
        print(f'Starting cross-validation for {fatigue}')
        y_ = {'Physical fatigue': y_phf, 'Mental fatigue': y_mf}[fatigue] # pick phF or MF

        # 1) outer loop: performance evaluation
        outer_cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED) if SHUFFLE \
            else StratifiedKFold(n_splits=folds)
        scores_cv = []
        for i, (train_outer_index, test_outer_index) in enumerate(outer_cv.split(X, y_)):
            print(f'fold: {i+1} / {folds}') # TODO: shuffle?

            # train/test split
            X_train, X_test = X[train_outer_index], X[test_outer_index]
            y_train, y_test = y_[train_outer_index], y_[test_outer_index]

            # normalize features (acc.to train set)
            if NORMALIZE_TRAIN:
                scaler = StandardScaler()
                scaler.fit(X_train)
                X_train = scaler.transform(X_train, copy=True)
                X_test = scaler.transform(X_test, copy=True)

            # 2) inner loop: hyperparameter tuning
            inner_cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED2) if SHUFFLE \
                else StratifiedKFold(n_splits=folds) # TODO: shuffle?
            # we test every combination of parameters by CV
            combinations = {}
            with tqdm(total=len(n_trees) * len(max_features) * len(max_depths) * \
                            len(criterions) * len(min_samples_splits),
                      file=sys.stdout) as pbar:
                for n_tree in n_trees:
                    for max_feature in max_features:
                        for max_depth in max_depths:
                            for criterion in criterions:
                                for min_sample_split in min_samples_splits:
                                    # model
                                    rf = RandomForestClassifier(n_estimators=n_tree,
                                                                criterion=criterion,
                                                                max_depth=max_depth,
                                                                min_samples_split=min_sample_split,
                                                                max_features=max_feature)

                                    # CV
                                    scores = cross_val_score(rf, X_train, y_train, cv=inner_cv, scoring='f1_weighted')

                                    # store score
                                    combination = (n_tree, max_feature, max_depth, criterion, min_sample_split)
                                    combinations[combination] = np.mean(scores)

                                    # for progress bar
                                    pbar.update(1)
                                    pbar.set_description(f"Inner loop - F1: {np.mean(scores)}")

            # best hyperparams
            best_combination, best_score = sorted(list(combinations.items()), key=lambda item: item[1])[-1]

            # for visualization
            print(f' Best F1 (inner): {best_score}')

            # use model with best hyperparams
            rf = RandomForestClassifier(n_estimators=best_combination[0],
                                        criterion=best_combination[3],
                                        max_depth=best_combination[2],
                                        min_samples_split=best_combination[4],
                                        max_features=best_combination[1])
            rf.fit(X_train, y_train)

            # evaluate
            y_pred = rf.predict(X_test)
            scores = evaluator(y_pred, y_test, verbose=False)
            scores_cv.append(scores)

            # for visualization
            print(f' Fold {i+1} F1 (outer): {scores["f1"]}')

        # final evaluation
        print('Performance model:')
        metrics = scores_cv[0].keys()
        for metric in metrics:
            # ignore confusion_matrix
            if metric == 'confusion':
                continue
            mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
            std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
            print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for Physical fatigue
fold: 1 / 5
Inner loop - F1: 0.7097199214982555:  34%|███▍      | 146/432 [00:10<00:21, 13.53it/s]



KeyboardInterrupt



### Leave-one-subject-out (LOSO)

In [19]:
'''
first scores without shuffling - check if the same for reproducability sake
0.776315820637198
0.6320400500625781
'''

'\nfirst scores without shuffling - check if the same for reproducability sake\n0.776315820637198\n0.6320400500625781\n'

In [20]:
subjects = [meta['subjectID'] for meta in metadata]
print(f'Subjects: {np.unique(subjects)}')
print(f'Total subjects: {len(np.unique(subjects))}')

Subjects: [ 3  4  5  7  9 10 12 13 14 15 16 17 19 20 21 22 23 24 25 26 27]
Total subjects: 21


In [ ]:
%%time
# nested CV
groups = subjects
folds_outer = len(np.unique(subjects))
folds_inner = 5

with warnings.catch_warnings():
    # ignore sklearn warning
    warnings.filterwarnings('ignore')

    for fatigue in ('Physical fatigue', 'Mental fatigue'):
        # load labels
        print(f'Starting cross-validation for {fatigue}')
        y_ = {'Physical fatigue': y_phf, 'Mental fatigue': y_mf}[fatigue] # pick phF or MF

        # 1) outer loop: performance evaluation
        outer_cv = LeaveOneGroupOut()
        scores_cv = []
        for i, (train_outer_index, test_outer_index) in enumerate(outer_cv.split(X, y_, groups)):
            print(f'fold: {i+1} / {folds_outer}')

            # train/test split
            X_train, X_test = X[train_outer_index], X[test_outer_index]
            y_train, y_test = y_[train_outer_index], y_[test_outer_index]

            # normalize features (acc.to train set)
            if NORMALIZE_TRAIN:
                scaler = StandardScaler()
                scaler.fit(X_train)
                X_train = scaler.transform(X_train, copy=True)
                X_test = scaler.transform(X_test, copy=True)

            # 2) inner loop: hyperparameter tuning
            inner_cv = StratifiedKFold(n_splits=folds_inner, shuffle=True, random_state=SEED2) if SHUFFLE \
                else StratifiedKFold(n_splits=folds_inner) # TODO: shuffle?
            # we test every combination of parameters by CV
            combinations = {}
            with tqdm(total=len(n_trees) * len(max_features) * len(max_depths) *
                            len(criterions) * len(min_samples_splits),
                      file=sys.stdout) as pbar:
                for n_tree in n_trees:
                    for max_feature in max_features:
                        for max_depth in max_depths:
                            for criterion in criterions:
                                for min_sample_split in min_samples_splits:
                                    # model
                                    rf = RandomForestClassifier(n_estimators=n_tree,
                                                                criterion=criterion,
                                                                max_depth=max_depth,
                                                                min_samples_split=min_sample_split,
                                                                max_features=max_feature)

                                    # CV
                                    scores = cross_val_score(rf, X_train, y_train, cv=inner_cv, scoring='f1_weighted')

                                    # store score
                                    combination = (n_tree, max_feature, max_depth, criterion, min_sample_split)
                                    combinations[combination] = np.mean(scores)

                                    # for progress bar
                                    pbar.update(1)
                                    pbar.set_description(f"Inner loop - F1: {np.mean(scores)}")

            # best hyperparams
            best_combination, best_score = sorted(list(combinations.items()), key=lambda item: item[1])[-1]

            # for visualization
            print(f' Best F1 (inner): {best_score}')

            # use model with best hyperparams
            rf = RandomForestClassifier(n_estimators=best_combination[0],
                                        criterion=best_combination[3],
                                        max_depth=best_combination[2],
                                        min_samples_split=best_combination[4],
                                        max_features=best_combination[1])
            rf.fit(X_train, y_train)

            # evaluate
            y_pred = rf.predict(X_test)
            scores = evaluator(y_pred, y_test, verbose=False)
            scores_cv.append(scores)

            # for visualization
            print(f' Fold {i+1} F1 (outer): {scores["f1"]}')

        # final evaluation
        print('Performance model:')
        metrics = scores_cv[0].keys()
        for metric in metrics:
            # ignore confusion_matrix
            if metric == 'confusion':
                continue
            mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
            std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
            print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for Physical fatigue
fold: 1 / 21
Inner loop - F1: 0.7247694888967076: 100%|██████████| 432/432 [15:37<00:00,  2.17s/it]
 Best F1 (inner): 0.7912698361342178
 Fold 1 F1 (outer): 0.1
fold: 2 / 21
Inner loop - F1: 0.721638915678327: 100%|██████████| 432/432 [15:26<00:00,  2.15s/it] 
 Best F1 (inner): 0.7780620568392249
 Fold 2 F1 (outer): 0.45714285714285713
fold: 3 / 21
Inner loop - F1: 0.7001474226234163: 100%|██████████| 432/432 [15:26<00:00,  2.15s/it]
 Best F1 (inner): 0.7796445072574159
 Fold 3 F1 (outer): 1.0
fold: 4 / 21
Inner loop - F1: 0.6974723739564298: 100%|██████████| 432/432 [15:28<00:00,  2.15s/it]
 Best F1 (inner): 0.7990758309968229
 Fold 4 F1 (outer): 1.0
fold: 5 / 21
Inner loop - F1: 0.7321471779440564: 100%|██████████| 432/432 [15:23<00:00,  2.14s/it]
 Best F1 (inner): 0.7721197442973308
 Fold 5 F1 (outer): 0.5333333333333333
fold: 6 / 21
Inner loop - F1: 0.7309843445976899: 100%|██████████| 432/432 [15:23<00:00,  2.14s/it]
 Best F1 (inner):